# Criar um Dataset Anotado Sintético com dados para avaliar LGPD


## Introdução


### Fontes

**GOLD**

- 2024 Artigo - Combining prompt‑based language models and weak supervision for labeling named entity recognition on legal documents
- 2024 Artigo - DODFMiner: An automated tool for Named Entity Recognition from Official Gazettes
- 2024 Artigo - Artigo - Legal Document Segmentation and Labeling Through Named Entity Recognition

**BRONZE**

- 2022 Artigo - Do We Still Need Human Assessors? Prompt-Based GPT-3 User Simulation in Conversational AI (...We come to the conclusion that, although classifiers trained on such synthetic data perform much better than random baselines)


### Post Medium - Pierre


Fonte: [Post Medium Pierre](https://medium.com/@pierre_guillou/nlp-modelos-e-web-app-para-reconhecimento-de-entidade-nomeada-ner-no-dom%C3%ADnio-jur%C3%ADdico-b658db55edfb)

- [Modelo HF](https://huggingface.co/pierreguillou/ner-bert-large-cased-pt-lenerbr)

Modelos de linguagem natural especializados no domínio jurídico brasileiro Para obter o modelo de linguagem natural especializado no domínio jurídico brasileiro, usamos o notebook e dataset a seguir:

- notebook: [Finetuning_language_model_BERtimbau_LeNER_Br.ipynb](https://github.com/piegu/language-models/blob/master/Finetuning_language_model_BERtimbau_LeNER_Br.ipynb)
- dataset: [pierreguillou/lener_br_finetuning_language_model](https://huggingface.co/datasets/pierreguillou/lener_br_finetuning_language_model)

Modelos NER especializados no domínio jurídico brasileiro Para obter o modelo NER especializado no domínio jurídico brasileiro, usamos o notebook e dataset a seguir:

- notebook: [HuggingFace_Notebook_token_classification_NER_LeNER_Br.ipynb](https://github.com/piegu/language-models/blob/master/HuggingFace_Notebook_token_classification_NER_LeNER_Br.ipynb)
- dataset: [lener_br](https://huggingface.co/datasets/lener_br)


[HF IOB](https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt)


## Fixando a SEED para reprodução


In [1]:
import random
import numpy as np
import torch

def fixando_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# ===
SEED = 42
fixando_seed(SEED)

## Carregar o Dataset LenerBR


In [2]:
# !pip install -U datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("lener_br", trust_remote_code=True)

In [4]:
# " ".join(dataset["train"]["tokens"][422])
# dataset["train"]["tokens"][422]
print(dataset["train"][422])


{'id': '422', 'tokens': ['106', ',', 'II', ',', "''", 'b', "''", ',', '151', 'e', '172', 'do', 'CTN', '-', 'os', 'quais', 'se', 'referem', 'à', 'tese', 'da', 'impossibilidade', 'de', 'remissão', 'e', 'de', 'suspensão', 'da', 'exigibilidade', 'dos', 'créditos', 'tributários', 'pelo', 'aludido', 'convênio', 'do', 'CONFAZ', '-', 'não', 'só', 'podem', 'como', 'devem', 'obrigatoriamente', 'ser', 'objeto', 'de', 'outra', 'demanda', ',', 'PODER', 'JUDICIÁRIO', 'DO', 'ESTADO', 'DO', 'ACRE', 'Segunda', 'Câmara', 'Cível', '13', 'Endereço', ':', 'Rua', 'Tribunal', 'de', 'Justiça', ',', 's/n', ',', 'Via', 'Verde', ',', 'CEP', '69.915-631', ',', 'Tel.', '68', '3302-0444/0445', ',', 'Rio', 'BrancoAC', '-', 'Mod', '.', '500244', '-', 'Autos', 'n.º', '0715337-93.2014.8.01.0001', 'porquanto', 'extrapolam', 'os', 'limites', 'objetivos', 'da', 'causa', 'de', 'pedir', 'inicial', '.'], 'ner_tags': [9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [5]:
def show_graph_labels(dataset, index):
    labels_model = [
        "O",
        "B-ORGANIZACAO",
        "I-ORGANIZACAO",
        "B-PESSOA",
        "I-PESSOA",
        "B-TEMPO",
        "I-TEMPO",
        "B-LOCAL",
        "I-LOCAL",
        "B-LEGISLACAO",
        "I-LEGISLACAO",
        "B-JURISPRUDENCIA",
        "I-JURISPRUDENCIA",
    ]
    words = dataset["train"][index]["tokens"]
    labels = dataset["train"][index]["ner_tags"]
    line1 = ""
    line2 = ""
    for word, label in zip(words, labels):
        full_label = labels_model[label]
        max_length = max(len(word), len(full_label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += full_label + " " * (max_length - len(full_label) + 1)
    print(line1)
    print(line2)


show_graph_labels(dataset, 422)


106          ,            II           ,            ''           b            ''           ,            151          e            172          do           CTN          - os quais se referem à tese da impossibilidade de remissão e de suspensão da exigibilidade dos créditos tributários pelo aludido convênio do CONFAZ        - não só podem como devem obrigatoriamente ser objeto de outra demanda , PODER JUDICIÁRIO DO ESTADO  DO      ACRE    Segunda       Câmara        Cível         13 Endereço : Rua     Tribunal de      Justiça , s/n , Via     Verde   , CEP 69.915-631 , Tel. 68 3302-0444/0445 , Rio     BrancoAC - Mod . 500244 - Autos            n.º              0715337-93.2014.8.01.0001 porquanto extrapolam os limites objetivos da causa de pedir inicial . 
B-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO I-LEGISLACAO O O  O     O  O       O O    O  O               O  O        O O  O 

## Carregar os dados sintéticos que serão inseridos


### Dados com Faker

- https://faker.readthedocs.io/en/master/#providers
- https://faker.readthedocs.io/en/master/locales/pt_BR.html


In [6]:
# !pip install Faker==25.2.0

In [7]:
def alterar_digito_verificador_cpf(cpf: str) -> str:
    cpf = list(cpf)
    cpf[12] = str((int(cpf[12]) + 1) % 10)
    return "".join(cpf)


alterar_digito_verificador_cpf("142.780.639-33")

'142.780.639-43'

In [10]:
from faker import Faker
import random


def generate_faker_sets(qnt=10, qnt_variavel=False, qnt_min=1):
    fake = Faker("pt_BR")

    date_pattern_br = "%d/%m/%Y"

    if qnt_variavel:
        qnt += 1

    qnt_final = random.randrange(qnt_min, qnt) if qnt_variavel else qnt
    qnt_metade_1 = round(qnt_final / 2)
    qnt_metade_2 = qnt_final - qnt_metade_1

    new_obj = {
        "NOME": [
            fake.name()
            for _ in range(random.randrange(qnt_min, qnt) if qnt_variavel else qnt)
        ],
        "DATA": [fake.date(pattern=date_pattern_br) for _ in range(qnt_metade_1)]
        + [f"{fake.month_name()} de {fake.year()}" for _ in range(qnt_metade_2)],
        "CPF": [
            alterar_digito_verificador_cpf(fake.cpf())
            for _ in range(random.randrange(qnt_min, qnt) if qnt_variavel else qnt)
        ],
        "TELEFONE": [
            fake.phone_number()
            for _ in range(random.randrange(qnt_min, qnt) if qnt_variavel else qnt)
        ],
        "EMAIL": [
            fake.email()
            for _ in range(random.randrange(qnt_min, qnt) if qnt_variavel else qnt)
        ],
        "DINHEIRO": [
            fake.pricetag().replace("R$", "R$ ")
            for _ in range(random.randrange(qnt_min, qnt) if qnt_variavel else qnt)
        ],
        "CEP": [
            fake.postcode()
            for _ in range(random.randrange(qnt_min, qnt) if qnt_variavel else qnt)
        ],
        "ENDERECO": [fake.state() for _ in range(qnt_metade_1)]
        + [fake.street_address() for _ in range(qnt_metade_2)],
    }

    return new_obj


# =========

# FAKE_DATA = generate_faker_sets(15)
FAKE_DATA = generate_faker_sets(2, True)
FAKE_DATA

{'NOME': ['Diego Rodrigues'],
 'DATA': ['julho de 1990'],
 'CPF': ['176.928.304-97', '748.230.691-06'],
 'TELEFONE': ['+55 (084) 9327 4098'],
 'EMAIL': ['pmonteiro@example.com', 'da-rosaemilly@example.org'],
 'DINHEIRO': ['R$ 33.781,10', 'R$ 34.811,21'],
 'CEP': ['95275020'],
 'ENDERECO': ['Rua de Pastor']}

In [9]:
def fake_data_formatted(fake_data={}):
    all_str = ""
    for k, v in fake_data.items():
        # if not v:
        #     continue
        v_rep = "; ".join(v).replace("\n", "; ")
        all_str += f"- {k}: {v_rep}\n"
    return all_str


# fake_data_formatted(FAKE_DATA)
fake_data_formatted(generate_faker_sets(2, True))

'- NOME: Carlos Eduardo Moraes; Ana Júlia Azevedo\n- DATA: agosto de 1978\n- CPF: 982.714.563-10\n- TELEFONE: +55 (081) 0761-5690\n- EMAIL: viniciusda-luz@example.org\n- DINHEIRO: R$ 0,73\n- CEP: 98280-701\n- ENDERECO: Vila Nascimento, 332\n'

### Gerando texto Prompt


In [10]:
# PROMPT_LLAMA_DADOS_SINTETICOS = fake_data_formatted(generate_faker_sets(2, True))
# PROMPT_LLAMA_DADOS_SINTETICOS = fake_data_formatted(generate_faker_sets(2))
# PROMPT_LLAMA_DADOS_SINTETICOS = generate_faker_sets(2)
# PROMPT_LLAMA_DADOS_SINTETICOS = generate_faker_sets(2, True)

# PROMPT_LLAMA_TEXTO = " ".join(dataset["train"]["tokens"][10])

# ============= MANUAL

PROMPT_LLAMA_TEXTO = f"""EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA"""
PROMPT_LLAMA_DADOS_SINTETICOS = {
    "NOME": ["Daniel Mendes"],
    "DATA": ["dezembro de 1990"],
    "CPF": ["490.183.567-10", "127.034.685-81"],
    "TELEFONE": ["0800 170 6459", "61 6556 4995"],
    "EMAIL": ["santosbarbara@example.net", "frezende@example.net"],
    "DINHEIRO": ["R$ 43,95", "R$ 3,58"],
    "CEP": ["28866-051", "29566719"],
    "ENDERECO": ["Praia Antônio Caldeira, 4"],
}

# =============

PROMPT_LLAMA = """Você é um especialista em Processamento de Linguagem Natural.
Você criará dados sintéticos.
Dado o TEXTO abaixo, insira todos os DADOS_SINTETICOS no TEXTO para aumentá-lo.
Não crie novos termos ou informações, apenas insira todos os DADOS_SINTETICOS no TEXTO.

DADOS_SINTETICOS: {dados_sinteticos}

TEXTO: "{texto}"

Resposta com o TEXTO aumentado:
"""
# ===============

exemplo_de_prompt = PROMPT_LLAMA.format(
    texto=PROMPT_LLAMA_TEXTO, dados_sinteticos=PROMPT_LLAMA_DADOS_SINTETICOS
)

print(exemplo_de_prompt)

Você é um especialista em Processamento de Linguagem Natural.
Você criará dados sintéticos.
Dado o TEXTO abaixo, insira todos os DADOS_SINTETICOS no TEXTO para aumentá-lo.
Não crie novos termos ou informações, apenas insira todos os DADOS_SINTETICOS no TEXTO.

DADOS_SINTETICOS: {'NOME': ['Daniel Mendes'], 'DATA': ['dezembro de 1990'], 'CPF': ['490.183.567-10', '127.034.685-81'], 'TELEFONE': ['0800 170 6459', '61 6556 4995'], 'EMAIL': ['santosbarbara@example.net', 'frezende@example.net'], 'DINHEIRO': ['R$ 43,95', 'R$ 3,58'], 'CEP': ['28866-051', '29566719'], 'ENDERECO': ['Praia Antônio Caldeira, 4']}

TEXTO: "EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA"

Resposta com o TEXTO aumentado:



### Carregando o Llama v3


In [11]:
from huggingface_hub import InferenceClient
import requests

URL_API = "http://127.0.0.1:8001"
MODEL_RESP_INFO = requests.get(f"{URL_API}/info").text

client = InferenceClient(model=URL_API)


def output_llama3_json(message):
    response = client.text_generation(
        message,
        max_new_tokens=512,
        stream=False,
        repetition_penalty=1,
        temperature=0.2,
        stop_sequences=[
            "<|start_header_id|>",
            "<|end_header_id|>",
            "<|eot_id|>",
            "\n\n",
        ],
        seed=SEED
    )
    return response


# ============
resposta_llama3 = output_llama3_json(exemplo_de_prompt)
print(resposta_llama3)

"EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA. O MINISTÉRIO PÚBLICO, por meio do DR. DANIEL MENDES, CPF 490.183.567-10, TELEFONE 0800 170 6459, EMAIL santosbarbara@example.net, foi notificado, por meio de carta registrada, com aviso de recebimento, na data de 15 de dezembro de 1990, na Rua Praia Antônio Caldeira, 4, nº 123, CEP 28866-051, Bairro: Centro, Município: São Paulo, Estado: São Paulo, e não compareceu à audiência, sendo considerado incumpridor do processo. O valor da causa é de R$ 43,95, e o valor da taxa de justiça é de R$ 3,58. O processo foi distribuído ao Juiz Dr. FREZENDE, CPF 127.034.685-81, TELEFONE 61 6556 4995, EMAIL frezende@example.net. O processo foi iniciado em 20 de dezembro de 1990, e a última atualização foi em 25 de dezembro d

In [12]:
def check_if_all_labels_are_in_the_text(text, labels={}, is_print_erros=True):
    text = text.lower()
    is_all_ok = True
    for k, v in labels.items():
        for label in v:
            if label.lower() not in text:
                if is_print_erros:
                    print(f"\nLabel '{label}' from {k} not found in the text")
                is_all_ok = False

    # print("All labels are in the text")
    return is_all_ok


# ==============
check_if_all_labels_are_in_the_text(resposta_llama3, PROMPT_LLAMA_DADOS_SINTETICOS)


Label '29566719' from CEP not found in the text


False

In [13]:
def remove_all_labels_are_not_in_the_text(text, labels={}):
    new_labels = {}
    for k, v in labels.items():
        new_labels[k] = [l for l in v if l in text]
    return new_labels


# ==============
new_labels = remove_all_labels_are_not_in_the_text(
    resposta_llama3, PROMPT_LLAMA_DADOS_SINTETICOS
)
new_labels

{'NOME': [],
 'DATA': ['dezembro de 1990'],
 'CPF': ['490.183.567-10', '127.034.685-81'],
 'TELEFONE': ['0800 170 6459', '61 6556 4995'],
 'EMAIL': ['santosbarbara@example.net', 'frezende@example.net'],
 'DINHEIRO': ['R$ 43,95', 'R$ 3,58'],
 'CEP': ['28866-051'],
 'ENDERECO': ['Praia Antônio Caldeira, 4']}

## Gerar dados Fakes Anotados com IOB tag


In [14]:
print("Qnt dataset: ", len(dataset["train"]))

Qnt dataset:  7828


In [15]:
# 1 texto = 10s
# 50 textos = ~9min 27s
DADOS_DATASET_DE = 0
DADOS_DATASET_QNT = 50

# =======

DADOS_DATASET_ATE = DADOS_DATASET_DE + DADOS_DATASET_QNT

OUTPUT_PATH = (
    f"./output/dataset_sintetico_{DADOS_DATASET_DE}_{DADOS_DATASET_ATE}.csv"
)
print("OUTPUT_PATH: ", OUTPUT_PATH)

OUTPUT_PATH:  ./output/dataset_sintetico_0_50.csv


In [16]:
def get_texto_dataset(dataset):
    texts = []
    for item in dataset["tokens"]:
        texts.append(" ".join(item))
    return texts


# ===============
DADOS_DATASET = get_texto_dataset(dataset["train"][DADOS_DATASET_DE:DADOS_DATASET_ATE])
print("Qnt: ", len(DADOS_DATASET))
DADOS_DATASET[:2]

Qnt:  50


['EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA .',
 '- O art . 178 , II , do CPC prescreve que compete ao Ministério Público intervir nas causas em que há interesses de incapazes , dispondo o art . 279 do mesmo diploma que o processo será nulo quando o Ministério Público não for intimado para acompanhar o feito em que deve intervir .']

In [17]:
def show_progress(index, total):
    progress = (index + 1) / total * 100
    print(f"\rProgresso: {progress:.2f}%", end="")


# =============
show_progress(1, 3)

Progresso: 66.67%

In [18]:
def inserir_dados_fake(dataset=[], qnt_fake_por_dados=2, qnt_fake_random=False):
    dados_fake = []
    dataset_len = len(dataset)

    for ind, texto in enumerate(dataset):
        try:
            dados_sinteticos = generate_faker_sets(qnt_fake_por_dados, qnt_fake_random)

            show_progress(ind, dataset_len)

            prompt_gerado = PROMPT_LLAMA.format(
                texto=texto, dados_sinteticos=dados_sinteticos
            )
            resposta_llama3 = output_llama3_json(prompt_gerado)

            is_all_labels_in = check_if_all_labels_are_in_the_text(
                text=resposta_llama3, labels=dados_sinteticos, is_print_erros=False
            )

            if not is_all_labels_in:
                # print(f"\nLabels não encontrados no texto índice: {ind} ...ajustando")
                dados_sinteticos = remove_all_labels_are_not_in_the_text(
                    resposta_llama3, dados_sinteticos
                )

            dados_fake.append(
                {"texto": resposta_llama3, "dados_sinteticos": dados_sinteticos}
            )
        except Exception as e:
            print("\nErro: ", e)
            print("\nprompt_gerado: ", prompt_gerado)
            print("\rresposta_llama3: ", resposta_llama3)
            # print("\nDados Gerados: ", dados_fake)
            continue

    return dados_fake


# =========

DADOS_FAKE = inserir_dados_fake(
    DADOS_DATASET,
    qnt_fake_por_dados=2,
    qnt_fake_random=True,
    # DADOS_DATASET,
    # qnt_fake_por_dados=1,
    # qnt_fake_random=False,
)
print("\nQnt: ", len(DADOS_FAKE))
DADOS_FAKE[:2]

Progresso: 100.00%
Qnt:  50


[{'texto': '"EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA. Luiz Felipe Aparecida, nascido em outubro de 1978, com CPF 568.701.324-63 e CPF 467.503.981-30, telefone (061) 5548 2351, e-mail jmartins@example.org e e-mail emoraes@example.net, possui R$ 8,01 e R$ 497,52 em dinheiro, e reside na Vila Breno Nunes, 85, CEP 88814648." . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .',
  'dados_sinteticos': {'NOME': ['Luiz Felipe Aparecida'],
   'DATA': ['outubro de 1978'],
   'CPF': ['568.701.324-63', '467.503.981-30'],
   'T

In [19]:
OUTPUT_PATH

'./output/dataset_sintetico_0_50.csv'

In [20]:
import pandas as pd

df = pd.DataFrame(DADOS_FAKE)
df.to_csv(OUTPUT_PATH, index=False)
print("Qnt: ", len(df))
df.head()

Qnt:  50


,texto,dados_sinteticos
0,"""EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO...","{'NOME': ['Luiz Felipe Aparecida'], 'DATA': ['..."
1,"- O art. 178, II, do CPC prescreve que compete...","{'NOME': ['Sra. Yasmin Borges', 'Maria Liz Ram..."
2,"""- Tratando-se de ação indenizatória ajuizada ...","{'NOME': ['Liz Cardoso'], 'DATA': ['julho de 2..."
3,- Tendo o vício sido arguido pelo Parquet nest...,"{'NOME': [], 'DATA': ['dezembro de 2011'], 'CP..."
4,"""- Preliminar acolhida para reconhecer a nulid...","{'NOME': ['Sophie Pinto'], 'DATA': ['junho de ..."
